In [1]:
!pip install icecream

In [2]:
from io import StringIO
import requests  # BASIC LIB FOR EXTRACTING DATA FROM APIS AND SERVERS ETC
from bs4 import BeautifulSoup # to provide us passing facilities of html files
import pandas as pd # for transformation of raw data
import sqlite3 # to load data
from datetime import datetime #kis time pr konsa log perform horha ya koi error aarha
from icecream import ic 

In [3]:
!pip install icecream

In [4]:
# step 0 
def log_progress(message):
    with open ("./code_log.txt", "a") as f:
        f.write(f"{datetime.now()}. {message}.\n")

In [5]:
# Step 1 
def extract(url,table_att):
    soup = requests.get(url).text
    web = BeautifulSoup(soup, "html.parser")
    table = web.find("span", string=table_att).find_next('table')
    df = pd.read_html(StringIO(str(table)))[0]
    log_progress("Data extracted successfully")
    return df

In [6]:
#step 2
def transformation(df, csv_path):
    exchange_rate = pd.read_csv(csv_path, index_col = 0).to_dict()['Rate']

    df['MC_GBP_Billion'] = round(df['Market cap (US$ billion)']*exchange_rate['GBP'],2)
    df['MC_EUR_Billion'] = round(df['Market cap (US$ billion)']*exchange_rate['EUR'],2)
    df['MC_INR_Billion'] = round(df['Market cap (US$ billion)']*exchange_rate['INR'],2)

    print(df['MC_EUR_Billion'][4])
    log_progress('Data Transformation Successfully done, step 2')

    return df

In [7]:
#STEP 3 LOAD
#CSV
def load_to_csv(df,out_path):
    df.to_csv(out_path)

    log_progress("Data loaded into the csv file")
    

In [11]:
#Step 3 
#SQLITE
def load_to_db(df,sql_conn, tn):
    df.to_sql(tn, sql_conn, if_exists= "replace", index= False)
    log_progress("Data loaded into the SQLITE Database")

In [12]:
if __name__ == '__main__':
    url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    output_csv_path = './Largest_banks_data.csv'
    database_name = 'Banks.db'
    table_name = 'Largest_banks'
    
    log_progress('Preliminaries complete. Initiating ETL process')

    df=(extract(url, 'By market capitalization'))
    log_progress('Extraction completeed')
    print(df)
    
    df = transformation(df, './exchange_rate.csv')
    log_progress('Data Transformed sucessfully')
    print(df)
    

   Rank                                Bank name  Market cap (US$ billion)
0     1                           JPMorgan Chase                    432.92
1     2                          Bank of America                    231.52
2     3  Industrial and Commercial Bank of China                    194.56
3     4               Agricultural Bank of China                    160.68
4     5                                HDFC Bank                    157.91
5     6                              Wells Fargo                    155.87
6     7                        HSBC Holdings PLC                    148.90
7     8                           Morgan Stanley                    140.83
8     9                  China Construction Bank                    139.82
9    10                            Bank of China                    136.81
146.86
   Rank                                Bank name  Market cap (US$ billion)  \
0     1                           JPMorgan Chase                    432.92   
1     2     

In [13]:
if __name__ == '__main__':
    url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    output_csv_path = './Largest_banks_data.csv'
    database_name = 'Banks.db'
    table_name = 'Largest_banks'
    
    log_progress('Preliminaries complete. Initiating ETL process')

    df=(extract(url, 'By market capitalization'))
    log_progress('Extraction completeed')
    print(df)
    
    df = transformation(df, './input/exchange_rate.csv')
    log_progress('Data Transformed sucessfully')
    print(df)

    load_to_csv(df, output_csv_path)
    log_progress('DataFrame has beenn loaded into csv')

    with sqlite3.connect(database_name) as conn:
        load_to_db(df, conn, table_name)
    log_progress('DataFrame has been uploaded into the SQlite Database Sucessfully')

   Rank                                Bank name  Market cap (US$ billion)
0     1                           JPMorgan Chase                    432.92
1     2                          Bank of America                    231.52
2     3  Industrial and Commercial Bank of China                    194.56
3     4               Agricultural Bank of China                    160.68
4     5                                HDFC Bank                    157.91
5     6                              Wells Fargo                    155.87
6     7                        HSBC Holdings PLC                    148.90
7     8                           Morgan Stanley                    140.83
8     9                  China Construction Bank                    139.82
9    10                            Bank of China                    136.81
146.86
   Rank                                Bank name  Market cap (US$ billion)  \
0     1                           JPMorgan Chase                    432.92   
1     2     